# Inheco Incubator Shaker


| Summary                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       | Photo                                            |
|-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|----------------------------------------------------|
| - [OEM Link](https://www.inheco.com/incubator-shaker.html)<br>- **Communication Protocol / Hardware**: Serial (FTDI)/ USB-A<br>- **Communication Level**: Firmware (documentation shared by OEM)<br>- same command set for: Incubator "MP", Incubator "DWP", Incubator Shaker "MP", Incubator Shaker "DWP" <br>- Incubator Shaker "MP" VID:PID 0403:6001<br>- Takes in a single plate via a loading tray, heats it to set temperature, shakes it to set RPM | ![quadrants](img/inheco_incubator_shaker_mp_dwp.png) |

This notebook provides a minimal working example to connect to and interact with an INHECO Incubator Shaker over a USB serial connection. It demonstrates how to initialize communication, issue basic commands, and interpret the device’s responses.

## About the Device and Setup

INHECO incubator shakers are modular devices used for temperature control and agitation in lab automation setups. Multiple devices can be connected in a stack via USB, each assigned a **device ID** and **subdevice number** (usually 0–5). Commands are sent via a serial protocol and must be carefully constructed with CRC checks and timeouts.

Ensure:
- The correct **serial port** is selected (e.g., `/dev/cu.usbserial-140` on macOS).
- You know the **device ID** for the incubator you're communicating with.
- Only one command is sent at a time — the device cannot queue or process multiple requests concurrently.

## Command Types

There are three classes of commands:

**Report Commands**

Used to **query** the device’s state or configuration. These include:
- `RFV0`: Read firmware version  
- `RFV2`: Read serial number  
- `RATx`: Read actual temperature (for subdevice `x`)

They return a string payload enclosed between header and terminator bytes.

**Set Commands**

Used to **configure** parameters or states. For example:
- `SCT020`: Set control temperature to 20°C  
- `SST1`: Set shaking speed to 1 Hz  

A successful response consists only of an acknowledgement terminator (`0x20 0x60`).

**Action Commands**

Used to **trigger actions**. Examples include:
- `AID`: Initialize device  
- `AOD`: Open lid  
- `ACD`: Close lid  

These may respond more slowly (up to 5 seconds), especially when mechanical motion is involved.

## Setup (Physical)

| Summary                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       | Photo                                            |
|-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|----------------------------------------------------|
| ![quadrants](img/inheco_incubator_shaker_physical_setup_overview.png) | |

## Setup (Programmatic)

In [1]:
import asyncio
import serial
import serial.tools.list_ports
import logging
from typing import Optional, List


class InhecoTask:
    """Represents a pending Inheco command awaiting a response."""
    def __init__(self, fut: asyncio.Future, cmd: str, timeout_time: float):
        self.fut = fut
        self.cmd = cmd
        self.timeout_time = timeout_time


class InhecoIncubatorShakerBackend:
    """Backend for controlling an Inheco Incubator Shaker via RS-232 or auto-detected FTDI."""

    def __init__(
        self,
        port: Optional[str] = None,
        dip_switch_id: int = 2,
        device: int = 0,
        write_timeout: float = 5.0,
        read_timeout: float = 10.0
    ):
        # --- Detect port automatically if not provided ---
        VID = "0403"
        PID = "6001"
        
        if port is None:
            for p in serial.tools.list_ports.comports():
                if f"{VID}:{PID}" in p.hwid:
                    port = p.device
                    # print(f"Detected Inheco device on {port} (VID:PID={VID}:{PID})")
                    break
            if port is None:
                raise RuntimeError(f"No FTDI ({VID}:{PID}) serial device found.")
        
        # Connection parameters
        self.port = port
        self.dip_switch_id = dip_switch_id
        self.device = device
        self.logger = logging.getLogger(__name__)

        # Serial connection
        self.ser = None
        self.write_timeout = write_timeout
        self.read_timeout = read_timeout

        # Cached machine status
        self.loading_tray = "unknown"
        self.incubator_type = "unknown"

    # === Lifecycle ===

    async def setup(self):
        self.ser = serial.Serial(
            port=self.port,
            baudrate=19200,
            bytesize=serial.EIGHTBITS,
            parity=serial.PARITY_NONE,
            stopbits=serial.STOPBITS_ONE,
            timeout=0
        )
        self.ser.write_timeout = self.write_timeout

        fw_version = await self.request_firmware_version()
        incubator_type = await self.request_incubator_type()
        serial_number = await self.request_serial_number()
        
        msg = (
            f"Connected to Inheco {incubator_type} on {self.port}\n"
            f"Machine serial number: {serial_number}\n"
            f"Firmware version: {fw_version}"
        )
        print(msg)
        self.logger.info(msg)

        await self.initialize()
        await self.close_lid()

    async def stop(self):
        if self.ser and self.ser.is_open:
            self.ser.close()
            self.logger.info("Disconnected from Inheco Incubator Shaker")

    # === Low-level I/O ===

    async def write(self, data: bytes):
        self.logger.debug(f"→ {data.hex()}")
        self.ser.write(data)

    async def _read_full_response(self, timeout: float = 5.0) -> bytes:
        start = asyncio.get_event_loop().time()
        buffer = bytearray()
        expected_header = 0xB0 + self.dip_switch_id
        full_tail = bytes([expected_header, 0x20, 0x60])

        while True:
            if self.ser.in_waiting:
                chunk = self.ser.read(self.ser.in_waiting)
                if chunk:
                    buffer.extend(chunk)
                    self.logger.debug(f"RX: {chunk.hex()}")

                    if b"\r" in buffer or b"\n" in buffer:
                        return bytes(buffer)
                    if buffer.endswith(full_tail):
                        return bytes(buffer)

            if asyncio.get_event_loop().time() - start > timeout:
                raise TimeoutError("Timed out waiting for complete response.")

            await asyncio.sleep(0.01)

    # === Encoding / Decoding ===

    def _crc8(self, data: bytearray) -> int:
        crc = 0xA1
        for byte in data:
            d = byte
            for _ in range(8):
                if (d ^ crc) & 1:
                    crc ^= 0x18
                    crc >>= 1
                    crc |= 0x80
                else:
                    crc >>= 1
                d >>= 1
        return crc & 0xFF

    def _build_message(self, command: str, device: int = 0) -> bytes:
        if not (0 <= device <= 5):
            raise ValueError("device must be between 0 and 5")
        full_command = f"T0{device}{command}"
        cmd = full_command.encode("ascii")
        length = len(cmd) + 3
        address = 0x30 + self.dip_switch_id
        proto = 0xC0 + len(cmd)
        message = bytearray([length, address, proto]) + cmd
        crc = self._crc8(message)
        return bytes(message + bytearray([crc]))

    def _is_report_command(self, command: str) -> bool:
        return command.upper().startswith("R")

    def _parse_response_bytes(self, response: bytes) -> str:
        """Decode and clean INHECO response bytes."""
        expected_header = 0xB0 + self.dip_switch_id
        long_tail = bytes([expected_header, 0x20, 0x60])
        short_tail = b"\r\n"
    
        if not response:
            raise ValueError("Empty response from device.")
        if not response.startswith(bytes([expected_header])) and not response[:1].isalpha():
            raise ValueError(f"Unexpected response header: {response[:1].hex()}")
        if not (response.endswith(long_tail) or response.endswith(short_tail)):
            raise ValueError(f"Unexpected response terminator: {response[-3:].hex()}")
    
        # Decode to ASCII and remove any control characters
        decoded = response.decode("ascii", errors="ignore")
    
        # Keep only printable ASCII and remove trailing garbage like '`'
        cleaned = "".join(ch for ch in decoded if 32 <= ord(ch) <= 126)
        cleaned = cleaned.rstrip(" `\t\r\n")
    
        return cleaned


    def _validate_acknowledgement(self, response: bytes):
        expected_header = 0xB0 + self.dip_switch_id
        if not response.startswith(bytes([expected_header])):
            raise ValueError(f"Unexpected ack header: {response[:1].hex()}")
        if not response.endswith(bytes([expected_header, 0x20, 0x60])):
            raise ValueError(f"Unexpected ack terminator: {response[-3:].hex()}")

    # === High-level Command API ===

    async def send_command(
        self,
        command: str,
        delay: float = 0.2,
        read_timeout: Optional[float] = None
    ) -> str:
        device = self.device
        msg = self._build_message(command, device=device)
        await self.write(msg)
        await asyncio.sleep(delay)
        response = await self._read_full_response(timeout=read_timeout or self.read_timeout)
        if not response:
            raise TimeoutError(f"No response from device for command: {command}")

        if self._is_report_command(command):
            return self._parse_response_bytes(response)
        else:
            self._validate_acknowledgement(response)
            return ""

    # === Public API ===

    # # # Report Commnads # # # 
    async def request_firmware_version(self):
        return await self.send_command("RFV0")

    async def request_serial_number(self):
        return await self.send_command("RFV2")

    async def request_incubator_type(self):
        """
        Identify what machine functionality you have:
        Incubator (1) with shaker, (2) MP or DWP chamber 
        """
        
        incubator_type_dict = {
            "0": "incubator_mp", # no shaker!
            "1": "incubator_shaker_mp",
            "2": "incubator_dwp", # no shaker!
            "3": "incubator_shaker_dwp",
        }
        
        resp = await self.send_command("RTS")

        incubator_identified = incubator_type_dict[resp]
        
        self.incubator_type = incubator_identified
        
        return incubator_identified

    async def request_plate_in_incubator(self):
        """ Request whether incubator currently hosts a plate """
        
        resp = await self.send_command("RLW")
    
        return bool(resp)

    async def request_last_calibration_date(self):
        """
        Reports the date and an alphanumeric string (e.g. operator) of the last calibration for the device.
        The Data is reported in the Format YYYY-MM-DD,xxxxx (Example: 2005-09-28,xxxxx).
        The five xs are alphanumeric wildcards.
        """
        resp = await incubator.send_command("RCM")
    
        return resp[:10]

    # # # Set Commands # # # 
    async def get_status(self):
        return await self.send_command("REE")    

    # # # Action Commands # # # 
    async def initialize(self):
        return await self.send_command("AID")

    # Loading tray commands #
    async def open_lid(self):
        await self.send_command("AOD")
        self.loading_tray = "open"

    async def close_lid(self):
        await self.send_command("ACD")
        self.loading_tray = "closed"




In [2]:
# Autodetect FTDI device
incubator = InhecoIncubatorShakerBackend(
    dip_switch_id=2,
    device=0
)
await incubator.setup()

Connected to Inheco incubator_shaker_mp on /dev/cu.usbserial-140
Machine serial number: 2013
Firmware version: IncShak_C_V3.50_04/2012


In [3]:
await incubator.request_plate_in_incubator()

True

In [4]:
incubator.loading_tray

'closed'

## Usage

### Testing & Investigation

In [9]:
await incubator.send_command("REE")

# Reports the status of the initialisation of the device and the status of the Lab-Ware detection
# if
# 0 Device initialised
# 1 Device not initialised
# 2 Lab-Ware status unknown
# 3 Device not initialised and Lab-Ware status unknow

'0'

'1'

In [16]:
for x in [
    0, 1, 2, 3, 
    5, 7, 10, 15,
    20, 21, 22, 23,
    24, 25
]:
    print(x, await incubator.send_command(f"RFV{x}") )

# Reports the date and an alphanumeric string (e.g. operator) of the last calibration for the device.
# The Data is reported in the Format YYYY-MM-DD,xxxxx (Example: 2005-09-28,xxxxx).
# The five xs are alphanumeric wildcards.

0 IncShak_C_V3.50_04/2012
1 BOOT_C_V3.12_05/2007
2 2013
3 0
5 COPYRIGHT INHECO
7 1
10 IncShak
15 350
20 325
21 315
22 IncShak_R_V3.25_12/2011
23 BOOT_R_V1.10_01/2007
24 IncShak_S_V3.15_05/2011
25 BOOT_S_V1.10_01/2007


'incubator_shaker_mp'

In [9]:
await incubator.stop()

In [10]:
await incubator.send_command("REE")

# Reports the status of the initialisation of the device and the status of the Lab-Ware detection
# if
# 0 Device initialised
# 1 Device not initialised
# 2 Lab-Ware status unknown
# 3 Device not initialised and Lab-Ware status unknow

PortNotOpenError: Attempting to use a port that is not open

### Loading Drawer

In [10]:
await incubator.open_lid()
# await asyncio.sleep(5)
# await incubator.close_lid()

In [11]:
await incubator.close_lid()

### Temperature Control

In [8]:
for t in range(5):
    temp_meas = {}
    
    for x in range(1,4):
        temp_meas[x] = await incubator.send_command(f"RAT{x}", read_timeout=60)

    print(t, temp_meas)
    time.sleep(1)

# This command reports the actual ambient or slot temperatures of the three sensors.
# It can be chosen whether the evaluated temperature or the sensor value should be reported.
# The temperatures are reported in 1/10 °C: 345 = 34,5 °C

0 {1: '172', 2: '173', 3: '173'}
1 {1: '172', 2: '173', 3: '173'}
2 {1: '172', 2: '173', 3: '173'}
3 {1: '173', 2: '173', 3: '173'}
4 {1: '172', 2: '173', 3: '173'}


In [ ]:
await incubator.send_command("RTT", read_timeout=60)


In [ ]:
await incubator.send_command("STT700", read_timeout=60)


### Shaking Control

TODO

In [4]:
import serial.tools.list_ports

def list_serial_devices():
    ports = serial.tools.list_ports.comports()
    for port in ports:
        print(f"Device: {port.device}")
        print(f"  Description: {port.description}")
        print(f"  HWID: {port.hwid}")
        print("-" * 40)

if __name__ == "__main__":
    print("Scanning USB serial devices...")
    list_serial_devices()


Scanning USB serial devices...
Device: /dev/cu.debug-console
  Description: n/a
  HWID: n/a
----------------------------------------
Device: /dev/cu.BoseQC35II
  Description: n/a
  HWID: n/a
----------------------------------------
Device: /dev/cu.Bluetooth-Incoming-Port
  Description: n/a
  HWID: n/a
----------------------------------------
Device: /dev/cu.usbserial-140
  Description: USB <-> Serial
  HWID: USB VID:PID=0403:6001 LOCATION=0-1.4
----------------------------------------


### Closing Connection

In [5]:
await incubator.stop()